## Testowanie działania klasyfikatorów na pisarzach Romantyzmu


Importujemy potrzebne biblioteki

In [1]:
import os, fnmatch
import pandas as pd

Dodajemy implementację potrzebnego klasyfikatora

In [3]:
class TxtDocMatrixClassifier:
    __unnecessary_chars = ["!",",",".","    ",":",";","?","(",")","—","*","»","…","«","—","-","„","”",'"',"°","'"] 
    __stops = ['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ', 'ani', 'az', 'aż', 'bardziej', 'bardzo', 'beda', 'bedzie', 'bez', 'deda', 'będą', 'bede', 'będę', 'będzie', 'bo', 'bowiem', 'by', 'byc', 'być', 'byl', 'byla', 'byli', 'bylo', 'byly', 'był', 'była', 'było', 'były', 'bynajmniej', 'cala', 'cali', 'caly', 'cała', 'cały', 'ci', 'cie', 'ciebie', 'cię', 'co', 'cokolwiek', 'cos', 'coś', 'czasami', 'czasem', 'czemu', 'czy', 'czyli', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokad', 'dokąd', 'dosc', 'dość', 'duzo', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dzis', 'dzisiaj', 'dziś', 'gdy', 'gdyby', 'gdyz', 'gdyż', 'gdzie', 'gdziekolwiek', 'gdzies', 'gdzieś', 'go', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych', 'iz', 'iż', 'ja', 'jak', 'jakas', 'jakaś', 'jakby', 'jaki', 'jakichs', 'jakichś', 'jakie', 'jakis', 'jakiś', 'jakiz', 'jakiż', 'jakkolwiek', 'jako', 'jakos', 'jakoś', 'ją', 'je', 'jeden', 'jedna', 'jednak', 'jednakze', 'jednakże', 'jedno', 'jego', 'jej', 'jemu', 'jesli', 'jest', 'jestem', 'jeszcze', 'jeśli', 'jezeli', 'jeżeli', 'juz', 'już', 'kazdy', 'każdy', 'kiedy', 'kilka', 'kims', 'kimś', 'kto', 'ktokolwiek', 'ktora', 'ktore', 'ktorego', 'ktorej', 'ktory', 'ktorych', 'ktorym', 'ktorzy', 'ktos', 'ktoś', 'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub', 'ma', 'mają', 'mało', 'mam', 'mi', 'miedzy', 'między', 'mimo', 'mna', 'mną', 'mnie', 'moga', 'mogą', 'moi', 'moim', 'moj', 'moja', 'moje', 'moze', 'mozliwe', 'mozna', 'może', 'możliwe', 'można', 'mój', 'mu', 'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych', 'natomiast', 'natychmiast', 'nawet', 'nia', 'nią', 'nic', 'nich', 'nie', 'niech', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'niz', 'niż', 'no', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod', 'podczas', 'pomimo', 'ponad', 'poniewaz', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza', 'prawie', 'przeciez', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'rowniez', 'również', 'sam', 'sama', 'są', 'sie', 'się', 'skad', 'skąd', 'soba', 'sobą', 'sobie', 'sposob', 'sposób', 'swoje', 'ta', 'tak', 'taka', 'taki', 'takie', 'takze', 'także', 'tam', 'te', 'tego', 'tej', 'ten', 'teraz', 'też', 'to', 'toba', 'tobą', 'tobie', 'totez', 'toteż', 'totobą', 'trzeba', 'tu', 'tutaj', 'twoi', 'twoim', 'twoj', 'twoja', 'twoje', 'twój', 'twym', 'ty', 'tych', 'tylko', 'tym', 'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej', 'wlasnie', 'właśnie', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'z', 'za', 'zaden', 'zadna', 'zadne', 'zadnych', 'zapewne', 'zawsze', 'ze', 'zeby', 'zeznowu', 'zł', 'znow', 'znowu', 'znów', 'zostal', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby']

    def __findBooksPaths(self) -> list:
        result = []
        for root, dirs, files in os.walk(self.__booksRelPath):
            for name in files:
                if fnmatch.fnmatch(name, '*_*.txt'):
                    result.append(os.path.join(root, name))
        return result 

    def __retrieveTokensFromBook(self, path) -> None:
        usidx = path.index('_', len(self.__booksRelPath)+1)
        author = path[len(self.__booksRelPath)+1:usidx]

        self.__authorsWB[author] = self.__authorsWB.get(author, dict())
        tokens_stop = []
        tokens = []

        with open(path, 'r', encoding='utf-8') as book:
            for line in book:        
                for char in self.__unnecessary_chars:
                    line = line.replace(char, '')
                line = line.rstrip().lower().split()

                if line:
                    tokens_stop.extend(line)

        for ts in tokens_stop:
            if not ts in self.__stops:
                tokens.append(ts)

        for word in sorted(tokens):
            if word.isalpha():
                self.__authorsWB[author][word] = self.__authorsWB[author].get(word, 0) + 1

    def __truncateAuthorsWB(self) -> None:
        for author in self.__authorsWB:
            tempWB = dict(sorted(self.__authorsWB[author].items(), key=lambda x: x[1], reverse = True))
            self.__authorsWB[author] = dict()

            count = min(len(tempWB.keys()), self.__lim)
            for token in tempWB:
                self.__authorsWB[author][token] = tempWB[token]

                if count < 0: break
                count -= 1
                
    def __buildAuthorsWB(self) -> None:
        paths = self.__findBooksPaths()
        for p in paths:
            self.__retrieveTokensFromBook(p)
        self.__truncateAuthorsWB()

    def __buildFragDict(self, frag) -> None:
        frag = frag.replace('\n', ' ')
        for char in self.__unnecessary_chars:
            frag = frag.replace(char, '')
        frag = frag.strip().lower().split()

        self.__fragTokens = dict()
        for token in frag:
            if token not in self.__stops:
                self.__fragTokens[token] = self.__fragTokens.get(token, 0) + 1
        self.__fragTokens = dict(sorted(self.__fragTokens.items(), key=lambda x: x[1], reverse = True))

    def __buildTDMfromFrag(self) -> None:
        self.__fragTDM = pd.DataFrame(columns=self.authors)
        for ft in self.__fragTokens:
            licznik = [0] * len(self.authors)
            for i, author in enumerate(self.authors):
                for token in self.__authorsWB[author]:
                    if ft == token:
                        licznik[i] += self.__authorsWB[author][token]
            element = pd.DataFrame([licznik], columns=self.authors, index=[ft])
            self.__fragTDM = self.__fragTDM.append(element)
        
    def classifyFullProb(self, frag) -> dict:
        self.__buildFragDict(frag)
        self.__buildTDMfromFrag()
        
        result = dict()
        for i, el in enumerate(self.__fragTDM.sum()):
            result[self.authors[i]] = el
            
        return result

    def classify(self, frag) -> str:
        full = self.classifyFullProb(frag)
        maxa = ""
        maxi = 0
        for author in full:
            if full[author] > maxi:
                maxi = full[author]
                maxa = author
        return maxa
    
    def getAuthors(self) -> list:
        return self.authors
    
#     def getAuthWB(self):
#         return self.__authorsWB
    
#     def getFragDict(self):
#         return self.__fragTokens

#     def getTDM(self):
#         return self.__fragTDM

    def __init__(self, bookRelPath, lim) -> None:
        self.__booksRelPath = bookRelPath
        self.__authorsWB = dict()
        self.__lim = lim

        self.__buildAuthorsWB()
        self.authors = list(self.__authorsWB.keys())


## Następnie przechodzimy do klasyfikacji i testowania klasyfikatora

Tworzymy klasifikator na podstawie, którego będziemy sprawdzać fragmenty tekstów.

In [4]:
K = TxtDocMatrixClassifier('..\\test_books\\romantyzm_books', 100)

Zaczniemy od wyszukania słów w miarę oczywistych i charakterystycznych dla danego autora.

In [15]:
#Framgent tesktu Coś ty atenom zrobił Sokratesie Norwida

fragment = '''Więc mniejsza o to, w jakiej spoczniesz urnie,
Gdzie, kiedy, w jakim sensie i obliczu,
Bo grób twój jeszcze odemkną powtórnie,
Inaczej będą głosić twe zasługi
I łez wylanych dziś będą się wstydzić,
A lać ci będą łzy potęgi drugiéj
Ci, co człowiekiem nie mogli cię widziéć.'''
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 0, 'aleksander-dumas-ojciec': 0, 'aleksander-dumas-syn': 0, 'alfred-de-musset': 0, 'alojzy-felinski': 13, 'aloysius-bertrand': 0, 'antanas-baranauskas': 0, 'autor-nieznany': 0, 'charles-baudelaire': 1, 'cyprian-kamil-norwid': 2, 'deotyma': 0, 'e-t-a-hoffmann': 0, 'edgar-allan-poe': 0, 'francois-rene-de-chateaubriand': 13, 'george-gordon-byron': 0, 'hans-christian-andersen': 0, 'honore-de-balzac': 0, 'jacob-i-wilhelm-grimm': 0, 'johann-wolfgang-von-goethe': 25, 'juliusz-slowacki': 0, 'karol-boloz-antoniewicz': 3, 'konstanty-gaszynski': 0, 'marceli-skalkowski': 0, 'stanislaw-jachowicz': 4, 'theodore-de-banville': 0, 'wincenty-pol': 0}


In [16]:
#Framgent tesktu Dziady Mickiewicza

fragment = '''DZIADY. Jest to nazwisko uroczystości obchodzonej dotąd między pospólstwem w wielu powiatach Litwy, Prus i Kurlandii, na pamiątkę dziadów, czyli w ogólności zmarłych przodków. Uroczystość ta początkiem swoim zasięga czasów pogańskich i zwała się niegdyś ucztą kozła, na której przewodniczył Koźlarz, Huslar, Guślarz, razem kapłan i poeta (gęślarz).'''
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 120, 'aleksander-dumas-ojciec': 90, 'aleksander-dumas-syn': 0, 'alfred-de-musset': 0, 'alojzy-felinski': 0, 'aloysius-bertrand': 0, 'antanas-baranauskas': 0, 'autor-nieznany': 0, 'charles-baudelaire': 0, 'cyprian-kamil-norwid': 0, 'deotyma': 127, 'e-t-a-hoffmann': 0, 'edgar-allan-poe': 6, 'francois-rene-de-chateaubriand': 0, 'george-gordon-byron': 38, 'hans-christian-andersen': 66, 'honore-de-balzac': 310, 'jacob-i-wilhelm-grimm': 5, 'johann-wolfgang-von-goethe': 35, 'juliusz-slowacki': 0, 'karol-boloz-antoniewicz': 0, 'konstanty-gaszynski': 0, 'marceli-skalkowski': 0, 'stanislaw-jachowicz': 4, 'theodore-de-banville': 0, 'wincenty-pol': 1}


In [17]:
#Framgent tesktu Panienka Z Okienka Deotyzma

fragemnt = '''Panowie i panny widocznie byli zadowoleni ze swojej strojności, bo szli z piersią wydętą jak u pawia, zerkając na przyglądające im się tłumy. Ale pan Kazimierz patrzył obojętnie; podobnych strojów, mniej lub więcej wykwintnych, mógł co dzień widywać krocie; podobny orszak weselny to w ludnym Gdańsku nie nowina. Dopiero za szeregami panien dostrzegł coś takiego, co go naprawdę zadziwiło, coś tak nadzwyczajnego, że oparł obie ręce o czekanik, aby wspiąć się na palce i lepiej zobaczyć.'''
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 120, 'aleksander-dumas-ojciec': 90, 'aleksander-dumas-syn': 0, 'alfred-de-musset': 0, 'alojzy-felinski': 0, 'aloysius-bertrand': 0, 'antanas-baranauskas': 0, 'autor-nieznany': 0, 'charles-baudelaire': 0, 'cyprian-kamil-norwid': 0, 'deotyma': 127, 'e-t-a-hoffmann': 0, 'edgar-allan-poe': 6, 'francois-rene-de-chateaubriand': 0, 'george-gordon-byron': 38, 'hans-christian-andersen': 66, 'honore-de-balzac': 310, 'jacob-i-wilhelm-grimm': 5, 'johann-wolfgang-von-goethe': 35, 'juliusz-slowacki': 0, 'karol-boloz-antoniewicz': 0, 'konstanty-gaszynski': 0, 'marceli-skalkowski': 0, 'stanislaw-jachowicz': 4, 'theodore-de-banville': 0, 'wincenty-pol': 1}


In [18]:
#Framgent tesktu Bak i Piłka Andersena

fragment = '''Pewnego dnia chłopczyk wziął piłkę z pudełka. Bąk widział, jak wysoko wzleciała w powietrze, jak ptak prawdziwy; w końcu zniknęła mu z oczu gdzieś w górze. Potem spadła na dół, a ile razy dotknęła ziemi, wzlatywała znowu, nie wiadomo, czy z tęsknoty do jaskółki, czy dlatego, że miała korek hiszpański wewnątrz.'''
print(K.classifyFullProb(fragment))

{'adam-mickiewicz': 83, 'aleksander-dumas-ojciec': 49, 'aleksander-dumas-syn': 52, 'alfred-de-musset': 0, 'alojzy-felinski': 3, 'aloysius-bertrand': 0, 'antanas-baranauskas': 0, 'autor-nieznany': 1, 'charles-baudelaire': 0, 'cyprian-kamil-norwid': 2, 'deotyma': 562, 'e-t-a-hoffmann': 0, 'edgar-allan-poe': 18, 'francois-rene-de-chateaubriand': 41, 'george-gordon-byron': 198, 'hans-christian-andersen': 517, 'honore-de-balzac': 316, 'jacob-i-wilhelm-grimm': 20, 'johann-wolfgang-von-goethe': 128, 'juliusz-slowacki': 209, 'karol-boloz-antoniewicz': 0, 'konstanty-gaszynski': 0, 'marceli-skalkowski': 1, 'stanislaw-jachowicz': 16, 'theodore-de-banville': 0, 'wincenty-pol': 0}


### Wnioski

W tych testach skupiliśmy się na działaniu klasyfikatora, jeżeli danymi testowymi były teksty z tej samej epoki u nas był to Romantyz. Taki test pokaże nam jak klasyfikacja działa w dosyć trudnych warunkach, ponieważ autorzy tej samej epoki, często posługują się podobnym językiem, składnią oraz tematyką. Dodatkowo jest ich całkem sporo w naszym folderze.  Po wykonaniu kilku sprawdzeń dla klasyfikatora możemy dojść do takich o to wniskówk:

* Klasyfikator o dziwo poradził sobie całkiem dobrze, nie we wszystkich przypadkach dobrze sklasifikował tekst, ale dla większości dobrze wybrał autora. 
* Co istotne wyniki są często bardzo zbiliżone co świadczy, tak jak wspomniałem o podobnym słownictwie oraz stylu pisania
* Mało było przypadków oczywsitych, gdzie jeden autor górował nad wszystkimi
* Co ciekawe nawet we fragmencie z Dziadów w którym pojawiło się bardzo charakterytstyczne słowo "DZIADY", został sklasifikowany Deotyma zamiast Mickiewicza, chociaż różnica była bardzo nieduża.